In [2]:
# %%writefile box.py
from copy import deepcopy

class Box:
    def __init__(self, x, y):
        self.coordinates = [(x, y), (x + 1, y), (x, y + 1), (x + 1, y + 1)]
        
        self.XY = (x, y)

        # lines
        self.TopLine = (self.coordinates[0], self.coordinates[1])
        self.LeftLine = (self.coordinates[0], self.coordinates[2])
        self.RightLine = (self.coordinates[1], self.coordinates[3])
        self.BottomLine = (self.coordinates[2], self.coordinates[3])
        # lines 
        self.lines = [self.TopLine, self.LeftLine, self.RightLine, self.BottomLine]

        # lines connection indicator 
        self._top = False
        self._left = False
        self._right = False
        self._bottom = False

        self.owner = None
        self.completed = False

        self.value = 1

    def connect(self, coordinates):
        line = coordinates
        success = False

        if line not in self.lines:
            return False
        
        if line == self.TopLine and self._top is False:
            self._top = True
            success = True
        elif line == self.LeftLine and self._left is False:
            self._left = True
            success = True
        elif line == self.RightLine and self._right is False:
            self._right = True
            success = True
        elif line == self.BottomLine and self._bottom is False:
            self._bottom = True
            success = True

        if self._top == True and self._bottom == True and self._left == True and self._right == True:
            self.completed = True
        
        return success

    def un_connect(self, coordinates):
        line = coordinates
        if line in self.lines:
            self.completed = False
            # self.owner = None

        if line == self.TopLine:
            self._top = False
        elif line == self.LeftLine:
            self._left = False
        elif line == self.RightLine:
            self._right = False
        elif line == self.BottomLine:
            self._bottom = False

        
    def copy(self):
        return deepcopy(self)
    
    def _repr_pretty_(self, p, cycle):
        if cycle:
            pass

        if self._top:
            p.text("*---*")
        else:
            p.text("*   *")
        p.break_()
        if self._left:
            p.text("|")
        else:
            p.text(" ")
        
        if self.completed:
            p.text(f" {self.owner} ")
        else:
            p.text("   ") 

        if self._right:
            p.text("|")

        p.break_()

        if self._bottom:
            p.text("*---*")
        else:
            p.text("*   *")



In [3]:
# %%writefile board.py
from collections import deque
from copy import deepcopy

class Board:
    display_single_box = False

    def __init__(self, m, n):
        self.player_score = 0
        self.ai_score = 0
        self.m = m
        self.n = n
        self._boxes = self._generate_boxes(m, n)
        self._open_vectors = self._generate_vectors(m, n)
        self._moves = []
        # Verificar si los vectores se generaron correctamente
        #print(f"Open Vectors Initialized: {self._open_vectors}")

    def _generate_boxes(self, rows, cols):
        """
        This function generates the boxes of the board
        """
        boxes = [[Box(x, y) for x in range(cols)] for y in range(rows)]

        return boxes
    
    def _generate_vectors(self, m, n):
        '''
        The vectors represent the available moves, or lines, which can
        be played on a game board of m rows and n columns. These are stored as tuples
        containing each coordinate and are stored in a queue. The vector queue, along
        with the list of boxes that correspond to the coordinates, are used to represent
        game state.
        Vector format: ((x1, y1), (x2, y2)).

        The vectors always point away from the origin (0, 0), so moving like (1, 0) => (0, 0)
        is not a valid move while (0, 0) => (1, 0) is a valid move
        '''
        vectors = set()
        for i in range(0, m + 1):
            for j in range(0, n):
                # Adding horizontal line vectors
                vectors.add(((j, i), (j + 1, i)))
                # Adding vertical line vectors if not in the last row
                if i < m:
                    vectors.add(((j, i), (j, i + 1)))
            # Adding the vertical line for the last column in the current row
            if i < m:
                vectors.add(((n, i), (n, i + 1)))
        return vectors

    def is_valid_move(self, coordinates):
        return coordinates in self._open_vectors

    
    def move(self, coordinates, player_move: bool = False, is_simulation: bool = False):
        if not self.is_valid_move(coordinates):
            print(f"Invalid move attempted: {coordinates}")
            return False

        if is_simulation:
            print(f"Simulating move: {coordinates} by {'Player' if player_move else 'AI'}")
        else:
            print(f"Attempting move: {coordinates} by {'Player' if player_move else 'AI'}")
        
        print(f"Available moves before: {self._open_vectors}")
        
        player = "P" if player_move else "A"
        
        self._open_vectors.remove(coordinates)
        self._moves.append(coordinates)
        closed = self._checkboxes(coordinates, player)
        
        print(f"Available moves after: {self._open_vectors}")
        print(f"Move resulted in closing box: {closed}")
        
        return closed



    def undo_last_move(self):
        if self._moves:
            move = self._moves.pop()
            self._open_vectors.add(move)
            self._undo_move_on_boxes(move)

    def _undo_move_on_boxes(self, move):
        for i in range(self.m):
            for j in range(self.n):
                box = self._boxes[i][j]
                if move in box.lines:
                    self._undo_box_completion(box, move)

    def _undo_box_completion(self, box, move):
        if box.completed:
            self._adjust_scores(box)
            box.owner = None
        box.un_connect(move)

    def _adjust_scores(self, box):
        if box.owner == "P":
            self.player_score -= 1
        else:
            self.ai_score -= 1               
        
    def has_moves(self):
        return bool(self._open_vectors)

    def get_available_moves(self):
        return self._open_vectors
        
    def _checkboxes(self, coordinates, player: str):
        closed = None
        for i in range(self.m):
            for j in range(self.n):
                box = self._boxes[i][j]
                if coordinates in box.lines:
                    box.connect(coordinates)
                if box.completed == True and box.owner == None:
                    box.owner = player
                    if player == "P":
                        self.player_score += 1
                    else:
                        self.ai_score += 1
                    closed = True
        return closed

    
    def copy(self):
        return deepcopy(self)
    
    def display_board(self):
        # Display player scores
        print(f"Player 1: {self.player_score}")
        print(f"Player AI: {self.ai_score}\n")

        for i in range(self.m):
            top_line, middle_line = self._generate_row_lines(i)
            print(top_line)
            print(middle_line)

        bottom_line = self._generate_bottom_line()
        print(bottom_line)
        print("")  # New line for spacing

    def _generate_row_lines(self, row):
        top_line = "   "  # Start with some spacing for alignment
        middle_line = "   "  # Line below to display vertical lines and boxes

        for j in range(self.n):
            top_line += self._generate_top_line_segment(j, row)
            middle_line += self._generate_middle_line_segment(j, row)

        # Last dot on the right end of the row
        top_line += "*"
        middle_line += "|" if ((self.n, row), (self.n, row + 1)) in self._moves else " "

        return top_line, middle_line

    def _generate_top_line_segment(self, col, row):
        segment = "*"
        if ((col, row), (col + 1, row)) in self._moves:
            segment += "---"
        else:
            segment += "   "
        return segment

    def _generate_middle_line_segment(self, col, row):
        segment = "| " if ((col, row), (col, row + 1)) in self._moves else "  "
        if self._boxes[col][row].completed:
            segment += f"{self._boxes[col][row].owner} "
        else:
            segment += "  "
        return segment

    def _generate_bottom_line(self):
        bottom_line = "   "
        for j in range(self.n):
            bottom_line += "*"
            if ((j, self.m), (j + 1, self.m)) in self._moves:
                bottom_line += "---"
            else:
                bottom_line += "   "
        bottom_line += "*"
        return bottom_line

    def _repr_pretty_(self, p, cycle):
        self.__display_single_box(p)


    def __display_single_box(self, p):
        for i in range(self.m):
            top_line, middle_line = self._generate_row_lines(i)
            p.text(top_line)
            p.break_()
            p.text(middle_line)
            p.break_()

        bottom_line = self._generate_bottom_line()
        p.text(bottom_line)
        p.break_()
        
    def validate_board_state(self):
        for i in range(self.m):
            for j in range(self.n):
                box = self._boxes[i][j]
                if box.completed:
                    for line in box.lines:
                        if line in self._open_vectors:
                            print(f"Inconsistency found: Closed box at ({i}, {j}) has an open line: {line}")

    def print_board_summary(self):
        print(f"Player Score: {self.player_score}, AI Score: {self.ai_score}")
        print(f"Open vectors: {self._open_vectors}")
        print(f"Moves made: {self._moves}")


In [4]:
Board.display_single_box = True

In [5]:
Board(3, 3)

   *   *   *   *
                
   *   *   *   *
                
   *   *   *   *
                
   *   *   *   *


In [6]:
# from board import Board

board = Board(2, 2)
board

   *   *   *
            
   *   *   *
            
   *   *   *


In [7]:
board.move(((0, 1), (1, 1)))
board

Attempting move: ((0, 1), (1, 1)) by AI
Available moves before: {((1, 1), (2, 1)), ((1, 0), (1, 1)), ((1, 0), (2, 0)), ((0, 0), (0, 1)), ((1, 1), (1, 2)), ((2, 1), (2, 2)), ((0, 2), (1, 2)), ((2, 0), (2, 1)), ((0, 1), (0, 2)), ((0, 1), (1, 1)), ((0, 0), (1, 0)), ((1, 2), (2, 2))}
Available moves after: {((1, 1), (2, 1)), ((1, 0), (1, 1)), ((1, 0), (2, 0)), ((0, 0), (0, 1)), ((1, 1), (1, 2)), ((2, 1), (2, 2)), ((0, 2), (1, 2)), ((2, 0), (2, 1)), ((0, 1), (0, 2)), ((0, 0), (1, 0)), ((1, 2), (2, 2))}
Move resulted in closing box: None


   *   *   *
            
   *---*   *
            
   *   *   *


In [8]:
board._boxes[0][0]

*   *
    
*---*

In [9]:
# %%writefile gamemanager.py
from copy import deepcopy
from typing import Optional
import random

class GameManager:
    
    def __init__(self, m, n, level, mode="minimax"):
        self._level = level
        self._board = Board(m, n)
        self.current_turn = "player"  # El turno inicial es del jugador

        if mode == "alphabeta":
            self._mode = self.alpha_beta
            print("Using AlphaBeta pruning")
        else:
            self._mode = self.mini_max
            print("Using Minimax")
            
            
    def _repr_pretty_(self, p, cycle):
        if (cycle):
            pass

        # Display player scores
        p.text(f"    AI Score : {self._board.ai_score}\n")
        p.text(f"Player Score : {self._board.player_score}\n")
        self._board._repr_pretty_(p, cycle)

    def get_victor(self):
        print("The game ended")
        print(f"Score: Player={self._board.player_score}, AI={self._board.ai_score}")

        if self._board.player_score > self._board.ai_score:
            return "player"
        elif self._board.player_score < self._board.ai_score:
            return "ai"
        else:
            return "draw"
    
    

    def get_move(self, origin, dest) -> tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]:
        coordinates = (origin, dest)

        while True:  # Ciclo para manejar turnos adicionales
            result = self._handle_turn(coordinates)
            if result is not None:
                return result

            if not self._board.has_moves():  # Verificar si el juego ha terminado
                return (None, self.get_victor())

    def _handle_turn(self, coordinates) -> Optional[tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]]:
        if self.current_turn == "player":
            return self._player_move(coordinates)
        elif self.current_turn == "ai":

            return self._ai_move()
        return None
                   
    def _player_move(self, coordinates, is_simulation: bool = False) -> Optional[tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]]:
        if is_simulation:
            # Crear una copia del estado del tablero para simular
            simulated_board = deepcopy(self._board)
            self.current_turn = "player"  # Cambiar el turno al jugador durante la simulación
            simulated_board.move(coordinates, player_move=True, is_simulation=True)
            self.current_turn = "ai"  # Después de la simulación, cambiar el turno a la IA
            return simulated_board  # Devolver el estado simulado

        # Movimiento real del jugador
        self.current_turn = "player"  # Asegurarse de que es el turno del jugador
        closed_box = self._board.move(coordinates, player_move=True)

        if closed_box:
            print("Player closed a box, gets another turn.")
            if not self._board.has_moves():
                return (None, self.get_victor())
            return None  # El jugador mantiene el turno si cierra una caja

        # Después del movimiento real del jugador, cambiar el turno a la IA
        self.current_turn = "ai"

        self._board.display_board()  # Mostrar el estado del tablero después del movimiento del jugador
        self._board.validate_board_state()  # Verificar consistencia del estado del tablero
        self._board.print_board_summary()  # Imprimir resumen del estado del tablero

        return (None, None)


    def _ai_move(self, is_simulation: bool = False) -> Optional[tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]]:
        if is_simulation:
            # Crear una copia del estado del tablero para simular
            simulated_board = deepcopy(self._board)
            eval, best_state = self._mode(simulated_board, self._level, True)
            return best_state
        

        # Movimiento real de la IA
        self.current_turn = "ai"  # Asegurarse de que es el turno de la IA

        # Obtener el mejor movimiento y el mejor estado simulado
        eval, best_move  = self._mode(self._board, self._level, True)
        
        
        if best_move is not None:
            # Aplica el mejor movimiento en el tablero real
            closed_box_ai = self._board.move(best_move, player_move=False, is_simulation=False)
            
            # Después del movimiento, cambiar el turno al jugador
            self.current_turn = "player"

            self._board.display_board()
            self._board.validate_board_state()
            self._board.print_board_summary()

            if closed_box_ai:
                print("AI closed a box, gets another turn.")
                if not self._board.has_moves():  # Corrected line
                    return (best_move, self.get_victor())
                return None

            return (best_move, None)

        return (None, None)


    def _find_best_move(self, current_state: Board, best_state: Board) -> Optional[tuple[tuple[int, int], tuple[int, int]]]:
        for move in current_state.get_available_moves():
            simulated_state, _ = self.simulate_move(current_state, move, "ai")
            print(f"Checking move: {move}")
            print(f"Simulated state: {simulated_state}")
            print(f"Best state: {best_state}")
            if simulated_state == best_state:
                return move
        print("No matching move found.")
        return None


    
    def evaluate(self, state: Board) -> float:
        ai_score = state.ai_score
        player_score = state.player_score

        total_lines = (state.m + 1) * state.n + state.m * (state.n + 1)
        remaining_lines = len(state._open_vectors)
        
        # Determinar la etapa del juego
        if remaining_lines > total_lines * 0.66:
            game_stage = "early"
        elif remaining_lines > total_lines * 0.33:
            game_stage = "mid"
        else:
            game_stage = "late"

        long_chains = 0

        # Analizar líneas y detectar cadenas
        chains = self._detect_chains(state)
        

        for chain in chains:
            if len(chain) >= 3:
                long_chains += 1
        
        # Evaluar el valor heurístico según la etapa del juego
        heuristic_value = ai_score - player_score

        if game_stage == "early":
            heuristic_value -= long_chains * 15  # Penaliza fuertemente cadenas largas en el inicio
        elif game_stage == "mid":
            heuristic_value -= long_chains * 10  # Penaliza pero menos en la mitad del juego
        else:
            heuristic_value += (len(chains) - long_chains) * 5  # Valora cerrar cadenas si están disponibles

        return heuristic_value

    def _detect_chains(self, state: Board):
        chains = []
        visited = set()

        for line in state._moves:
            if line not in visited:
                chain = self._explore_chain(state, line, visited)
                if chain:
                    chains.append(chain)

        return chains

    def _explore_chain(self, state: Board, start_line, visited: set):
        chain = []
        stack = [start_line]

        while stack:
            line = stack.pop()
            if line in visited:
                continue

            visited.add(line)
            chain.append(line)

            connected_lines = self._get_connected_lines(state, line, visited)
            stack.extend(connected_lines)

        return chain

    def _get_connected_lines(self, state: Board, line, visited: set):
        connected_lines = []

        for box in state._boxes:
            for b in box:
                if line in b.lines:
                    for l in b.lines:
                        if l != line and l in state._moves and l not in visited:
                            connected_lines.append(l)

        return connected_lines

    @staticmethod
    def simulate_move(state, move, current_turn):
        
        # Realiza una copia profunda del estado para no modificar el original
        simulated_state = deepcopy(state)
        
        # Aplica el movimiento para el jugador actual
        simulated_state.move(move, player_move=(current_turn == "player"), is_simulation=True)
        
        # Cambia el turno para la siguiente simulación
        next_turn = "player" if current_turn == "ai" else "ai"
        
        # Retorna el estado simulado y el próximo turno
        return simulated_state, next_turn


    def mini_max(self, state: Board, ply: int, max_min: bool) -> tuple[float, Board]:
        if ply == 0 or len(state._open_vectors) == 0:
            h = self.evaluate(state)
            return (h, None)
        
        available_moves = list(state.get_available_moves())

        if max_min:
            return self._maximize(state, available_moves, ply)
        else:
            return self._minimize(state, available_moves, ply)


    def _maximize(self, state: Board, available_moves: list, ply: int) -> tuple[float, tuple[int, int]]:
        max_val = float('-inf')
        best_move = None

        for move in available_moves:
            # Simula el movimiento en una copia del tablero
            simulated_state, _ = self.simulate_move(state, move, "ai")
            
            # Evalúa el estado resultante mediante una llamada recursiva a minimax
            eval, _ = self.mini_max(simulated_state, ply - 1, False)
            
            # Si el valor es mejor, guarda el movimiento como el mejor
            if eval > max_val:
                max_val = eval
                best_move = move  # Solo guardas el mejor movimiento

        return (max_val, best_move)


    def _minimize(self, state: Board, available_moves: list, ply: int) -> tuple[float, tuple[int, int]]:
        min_val = float('inf')
        best_move = None

        for move in available_moves:
            # Simula el movimiento en una copia del tablero
            simulated_state, _ = self.simulate_move(state, move, "player")
            
            # Evalúa el estado resultante mediante una llamada recursiva a minimax
            eval, _ = self.mini_max(simulated_state, ply - 1, True)
            
            # Si el valor es mejor, guarda el movimiento como el mejor
            if eval < min_val:
                min_val = eval
                best_move = move  # Solo guardas el mejor movimiento

        return (min_val, best_move)


    def alpha_beta(self, state: Board, ply: int, is_max: bool, alpha=float('-inf'), beta=float('inf')) -> tuple[float, Board]:
        if ply == 0 or not state.has_moves():
            h = self.evaluate(state)
            return (h, None)

        available_moves = list(state.get_available_moves())

        if is_max:
            return self._maximize_alpha_beta(state, available_moves, ply, alpha, beta)
        else:
            return self._minimize_alpha_beta(state, available_moves, ply, alpha, beta)

    def _maximize_alpha_beta(self, state: Board, available_moves: list, ply: int, alpha: float, beta: float) -> tuple[float, tuple[int, int]]:
        max_val = float('-inf')
        best_move = None

        for move in available_moves:
            # Simula el movimiento en una copia del tablero
            simulated_state, _ = self.simulate_move(state, move, "ai")
            
            # Evalúa el estado resultante mediante una llamada recursiva a alpha_beta
            eval, _ = self.alpha_beta(simulated_state, ply - 1, False, alpha, beta)
            
            # Si el valor es mejor, guarda el movimiento como el mejor
            if eval > max_val:
                max_val = eval
                best_move = move  # Solo guardas el mejor movimiento

            # Actualiza alfa
            alpha = max(alpha, max_val)
            
            # Poda alfa-beta
            if beta <= alpha:
                break

        return (max_val, best_move)



    def _minimize_alpha_beta(self, state: Board, available_moves: list, ply: int, alpha: float, beta: float) -> tuple[float, tuple[int, int]]:
        min_val = float('inf')
        best_move = None

        for move in available_moves:
            # Simula el movimiento en una copia del tablero
            simulated_state, _ = self.simulate_move(state, move, "player")
            
            # Evalúa el estado resultante mediante una llamada recursiva a alpha_beta
            eval, _ = self.alpha_beta(simulated_state, ply - 1, True, alpha, beta)
            
            # Si el valor es mejor, guarda el movimiento como el mejor
            if eval < min_val:
                min_val = eval
                best_move = move  # Solo guardas el mejor movimiento

            # Actualiza beta
            beta = min(beta, min_val)
            
            # Poda alfa-beta
            if beta <= alpha:
                break

        return (min_val, best_move)



In [10]:
# game_min = GameManager(10, 10, 3)
# print(game_min._mode)
# game_min

In [11]:
# game_alpha = GameManager(10, 10, 10, mode="alphabeta")
# print(game_alpha._mode)
# game_alpha

In [12]:
# move = game_min.get_move((0, 0), (0, 1))
# print(move)
# game_min

In [13]:
# move = game_alpha.get_move((0, 0), (0, 1))
# print(move)
# game_alpha

In [14]:
# import random
# class Tester:
#     def __init__(self, board_size=(4, 4), level=3, mode="minimax"):
#         self.board_size = board_size
#         self.level = level
#         self.mode = mode
#         self.game_manager = GameManager(board_size[0], board_size[1], level, mode)
    
#     def simulate_game(self, player_starts=True):
#         print("\nStarting a new game...")
#         self.game_manager = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
#         current_turn = "player" if player_starts else "ai"

#         while self.game_manager._board.has_moves():
#             print(f"\nCurrent Turn: {current_turn.upper()}")
            
#             if current_turn == "player":
#                 # Movimiento aleatorio del jugador (puedes cambiar esto por una entrada manual si lo deseas)
#                 move = random.choice(list(self.game_manager._board.get_available_moves()))
#                 _, result = self.game_manager.get_move(*move)
#                 current_turn = "ai"
#             else:
#                 # Movimiento de la IA
#                 move, result = self.game_manager.get_move(*random.choice(list(self.game_manager._board.get_available_moves())))
#                 current_turn = "player"
            
#             # Imprimir el tablero y la evaluación heurística después de cada movimiento
#             self.print_board_and_heuristic()
            
#             if result:
#                 break
        
#         if result:
#             print("\nThe game ended")
#             print(f"Final Score: Player={self.game_manager._board.player_score}, AI={self.game_manager._board.ai_score}")
#             return self.game_manager.get_victor()
#         return None

#     def print_board_and_heuristic(self):
#         # Imprime el tablero actual
#         self.game_manager._board.display_board()
        
#         # Calcula e imprime la evaluación heurística del estado actual
#         heuristic_value = self.game_manager.evaluate(self.game_manager._board)
#         print(f"Heuristic Value: {heuristic_value}\n")

#     def analyze_performance(self, num_games=50):
#         results = {"ai_wins": 0, "player_wins": 0, "draws": 0}

#         for i in range(num_games):
#             print(f"Starting game {i + 1}...")
#             victor = self.simulate_game(player_starts=random.choice([True, False]))
#             if victor == "ai":
#                 results["ai_wins"] += 1
#             elif victor == "player":
#                 results["player_wins"] += 1
#             else:
#                 results["draws"] += 1
#             print(f"Game {i + 1} ended. Winner: {victor}")
#             print("-" * 30)

#         print("\nFinal Results after {} games:".format(num_games))
#         print("AI Wins: {}".format(results["ai_wins"]))
#         print("Player Wins: {}".format(results["player_wins"]))
#         print("Draws: {}".format(results["draws"]))

#         return results


In [15]:
# class TesterAIvsAI:
#     def __init__(self, board_size=(4, 4), level=3, mode="alphabeta"):
#         self.board_size = board_size
#         self.level = level
#         self.mode = mode
#         self.game_manager_maximizer = GameManager(board_size[0], board_size[1], level, mode)
#         self.game_manager_minimizer = GameManager(board_size[0], board_size[1], level, mode)
    
#     def simulate_competing_ais(self, maximizing_starts=True):
#         print("\nStarting a new AI vs AI game...")
#         gm_max = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
#         gm_min = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
#         current_turn = "maximizer" if maximizing_starts else "minimizer"

#         while gm_max._board.has_moves():
#             print(f"\nCurrent Turn: {current_turn.upper()}")
            
#             if current_turn == "maximizer":
#                 move, _ = gm_max.get_move(*random.choice(list(gm_max._board.get_available_moves())))
#                 current_turn = "minimizer"
#             else:
#                 move, _ = gm_min.get_move(*random.choice(list(gm_min._board.get_available_moves())))
#                 current_turn = "maximizer"
            
#             # Imprimir el tablero después de cada movimiento
#             gm_max._board.display_board()
#             print(f"Maximizer Heuristic Value: {gm_max.evaluate(gm_max._board, maximize=True)}")
#             print(f"Minimizer Heuristic Value: {gm_min.evaluate(gm_min._board, maximize=False)}\n")

#         print("\nThe game ended")
#         print(f"Final Score: Maximizer={gm_max._board.ai_score}, Minimizer={gm_max._board.player_score}")
#         return gm_max.get_victor()

#     def analyze_ai_vs_ai(self, num_games=10):
#         results = {"maximizer_wins": 0, "minimizer_wins": 0, "draws": 0}

#         for i in range(num_games):
#             print(f"Starting AI vs AI game {i + 1}...")
#             victor = self.simulate_competing_ais(maximizing_starts=random.choice([True, False]))
#             if victor == "ai":
#                 results["maximizer_wins"] += 1
#             elif victor == "player":
#                 results["minimizer_wins"] += 1
#             else:
#                 results["draws"] += 1
#             print(f"AI vs AI Game {i + 1} ended. Winner: {victor}")
#             print("-" * 30)

#         print(f"\nFinal Results after {num_games} AI vs AI games:")
#         print("Maximizer Wins: {}".format(results["maximizer_wins"]))
#         print("Minimizer Wins: {}".format(results["minimizer_wins"]))
#         print("Draws: {}".format(results["draws"]))

#         return results


In [16]:
# # Inicializar el tester
# tester = Tester(board_size=(4, 4), level=3, mode="minimax")

# # Simular un juego con impresión de cada movimiento
# victor = tester.simulate_game(player_starts=True)
# print(f"\nGame Over! The winner is: {victor}")

In [17]:
# # Inicializar el tester
# tester = Tester(board_size=(4, 4), level=3, mode="minimax")

# # Simular y analizar el rendimiento de la IA en 50 juegos
# performance_results = tester.analyze_performance(num_games=20)

In [18]:
# # Para usar el nuevo tester de IA vs IA:
# tester_ai_vs_ai = TesterAIvsAI(board_size=(4, 4), level=3, mode="alphabeta")
# victor_ai = tester_ai_vs_ai.simulate_competing_ais(maximizing_starts=True)
# performance_ai_vs_ai = tester_ai_vs_ai.analyze_ai_vs_ai(num_games=10)

In [41]:
import random

class Tester:
    def __init__(self, board_size=(4, 4), level=3, mode="minimax"):
        self.board_size = board_size
        self.level = level
        self.mode = mode
        self.game_manager = GameManager(board_size[0], board_size[1], level, mode)
    
    def simulate_random_player_vs_ai(self):
        print("\nStarting a new game: Player (random) vs AI")
        self.game_manager = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
        print(f"Player starts the game!")

        while self.game_manager._board.has_moves():
            if self.game_manager.current_turn == "player":
                # Movimiento aleatorio del jugador
                move = random.choice(list(self.game_manager._board.get_available_moves()))
                print(f"\nPlayer's turn. Attempting move: {move}")  # Imprimir las coordenadas del movimiento del jugador
                self.game_manager.get_move(*move)
            else:
                print("\nAI's turn.")
                move, _ = self.game_manager.get_move(None, None)  # La IA toma su turno
                if move is not None:
                    print(f"AI attempts move: {move}")  # Imprimir las coordenadas del movimiento de la IA
                    heuristic_value = self.game_manager.evaluate(self.game_manager._board)
                    print(f"Heuristic Value after AI move: {heuristic_value}")
            
            # Imprimir puntuaciones después de cada movimiento
            self.print_scores()

        victor = self.game_manager.get_victor()
        print(f"\nGame Over! The winner is: {victor}")
        return victor

    def simulate_ai_vs_ai(self):
        print("\nStarting a new game: AI vs AI")
        self.game_manager = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
        print(f"AI ({self.mode}) starts the game!")
        self.game_manager.current_turn = "ai"  # Forzar que inicie la IA

        while self.game_manager._board.has_moves():
            if self.game_manager.current_turn == "ai":
                print(f"\nAI's turn using {self.mode} strategy.")
                move, _ = self.game_manager.get_move(None, None)
                if move is not None:
                    print(f"AI attempts move: {move}")  # Imprimir las coordenadas del movimiento de la IA
                    heuristic_value = self.game_manager.evaluate(self.game_manager._board)
                    print(f"Heuristic Value after AI move: {heuristic_value}")
            else:
                move = random.choice(list(self.game_manager._board.get_available_moves()))
                print(f"Opponent AI attempts random move: {move}")  # Imprimir las coordenadas del movimiento de la IA oponente
                self.game_manager.get_move(*move)
            
            # Imprimir puntuaciones después de cada movimiento
            self.print_scores()
        
        victor = self.game_manager.get_victor()
        print(f"\nGame Over! The winner is: {victor}")
        return victor

    def analyze_performance(self, num_games=10, mode="random_vs_ai"):
        results = {"player_wins": 0, "ai_wins": 0, "draws": 0}

        for i in range(num_games):
            if mode == "random_vs_ai":
                print(f"\nSimulating Game {i+1} - Random Player vs AI")
                victor = self.simulate_random_player_vs_ai()
            elif mode == "ai_vs_ai":
                print(f"\nSimulating Game {i+1} - AI vs AI")
                victor = self.simulate_ai_vs_ai()

            if victor == "player":
                results["player_wins"] += 1
            elif victor == "ai":
                results["ai_wins"] += 1
            else:
                results["draws"] += 1

        print(f"\nResults after {num_games} games ({mode}):")
        print(f"Player Wins: {results['player_wins']}")
        print(f"AI Wins: {results['ai_wins']}")
        print(f"Draws: {results['draws']}")
        return results

    def play_human_vs_ai(self):
        print("\nStarting a new game: Human vs AI")
        self.game_manager = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
        print(f"You start the game!")

        while self.game_manager._board.has_moves():
            if self.game_manager.current_turn == "player":
                # Movimiento del jugador
                move = self.get_human_move()
                self.game_manager.get_move(*move)
            else:
                print("\nAI's turn.")
                move, _ = self.game_manager.get_move(None, None)  # La IA toma su turno
                if move is not None:
                    print(f"AI attempts move: {move}")  # Imprimir las coordenadas del movimiento de la IA
                    heuristic_value = self.game_manager.evaluate(self.game_manager._board)
                    print(f"Heuristic Value after AI move: {heuristic_value}")
            
            # Imprimir puntuaciones después de cada movimiento
            self.print_scores()

        victor = self.game_manager.get_victor()
        print(f"\nGame Over! The winner is: {victor}")
        return victor

    def get_human_move(self):
        while True:
            try:
                print("\nYour turn. Please enter your move:")
                start_x = int(input("Enter start x coordinate (e.g., 0): "))
                start_y = int(input("Enter start y coordinate (e.g., 0): "))
                end_x = int(input("Enter end x coordinate (e.g., 1): "))
                end_y = int(input("Enter end y coordinate (e.g., 0): "))

                move = ((start_x, start_y), (end_x, end_y))
                
                if self.game_manager._board.is_valid_move(move):
                    return move
                else:
                    print("Invalid move. Please try again.")
            except ValueError:
                print("Invalid input. Please enter integer coordinates.")

    def print_scores(self):
        print(f"\nCurrent Scores:")
        print(f"You: {self.game_manager._board.player_score}")
        print(f"AI: {self.game_manager._board.ai_score}")

    
    def print_scores(self):
        print(f"\nCurrent Scores:")
        print(f"Player: {self.game_manager._board.player_score}")
        print(f"AI: {self.game_manager._board.ai_score}")


In [42]:
# Inicializar el tester
tester = Tester(board_size=(4, 4), level=2, mode="alphabeta")

# Simular un juego con jugador aleatorio vs IA
#tester.simulate_random_player_vs_ai()
tester.play_human_vs_ai()
# # Simular un juego de IA vs IA
#tester.simulate_ai_vs_ai()

# # Analizar el rendimiento en múltiples juegos
# performance_results = tester.analyze_performance(num_games=10)


Using AlphaBeta pruning

Starting a new game: Human vs AI
Using AlphaBeta pruning
You start the game!

Your turn. Please enter your move:
Invalid move. Please try again.

Your turn. Please enter your move:
Invalid move. Please try again.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinates.

Your turn. Please enter your move:
Invalid input. Please enter integer coordinate